# Homework 1

The learning goals of this first hands-on sheet are: 
* to make sure that you can execute code on your machines or on Google Colab in order to experiment with LMs and RL yourself!
* to familiarize yourself with the [HuggingFace library](https://medium.com/nlplanet/two-minutes-nlp-beginner-intro-to-hugging-face-main-classes-and-functions-fb6a1d5579c4) which provides many pretrained LMs and handy tools for working with them,
* to develop basic intuitions about core RL concepts,
* and to train your first RL agent!

Most importantly, the homework is intended to showcase important practical aspects, provide space for learning how to find solutions to practical problems, and further conceptual understanding of the topics we discuss in class. It is *not* meant to dismay you. Therefore, even if you don't have a lot of ML / programming / technical background, you are warmly encouraged to take on the tasks, ask questions and discuss any concerns you have (with fellow students or me). There are also some hints and links to resources throughout the tasks which may help you get information which will help solving the tasks. 

### Homework logistics

* You will have two weeks to complete the assignment (until Wed, November 8th, 12:30pm).
* **Please do and submit your homework by yourself!**
* However, you are warmly encouraged to ask questions and help each other, without posting full solutions, via active discussions in the dedicated Forum space on Moodle ("Homework 1"). Most active participants of the Forum discussions will earn some extra points for their grade!
* Please submit your solutions via Moodle. You will find a quiz called "Homework 1" with questions and answer fields corresponding the respective exercise numbers listed below. 
* If you have questions or difficulties with the homework, please try to solve them with the help of your fellow students via Forum. However, I will also offer a **consultation session on Tuesday, October 31st, 2pm-4pm, on Zoom**, under the usual class link. Also don't hesitate to reach out to me via email if you have any questions, struggle or feel overwhelmed.

## Preliminaries

The exercises below will require you to execute Python code. You can do so either on your own machine, or by using [Google Colab](https://colab.research.google.com/) (free, only requires a Google account). You can easily do the latter by pressing the Colab icon at the top of the webook's page.
You are encouraged to use the Colab option to avoid complications with local package installations etc.
To speed up the execution of the code on Colab (especially Exercise 1), you can use the available GPU. For that, before executing your code, navigate to Runtime > Change runtime type > GPU > Save.

However, if you do want to run the code locally on your machine, I strongly encourage you to create an environment (e.g., with Conda) before you install any dependencies, and please keep in mind that pretrained language model weights might take up quite a bit of space on your hard drive or might require high RAM for prediction. In particular, the model used in these exercises requires 6GB disc space and **around 8GB RAM** for stable training.

Note that the class uses PyTorch. For those of you who wish to complete final projects which include programming, you are also free to use TensorFlow for that (but I may be able to provide less support with that).

## Exercise 1 (20 points)

In this exercise, we will load a pretrained LM from HuggingFace and explore how to work with it, using the tools provided by the library.

### Exercise 1.1 (5 points)

Your task is to use the pretrained model "GPT-NEO" (1.3B parameters) to *run inference*. In particular, your task is to complete the code below in order to produce a continuation for the sentence "Reinforcement learning is " using **beam search with k=5**. (Hint: beam-search is a particular *decoding scheme* used on top of trained language models. If you are not familiar with it, please do some research to get an overall idea about it as part of this task.) 

You can find information for completing the code, e.g., [here](https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TextGenerationPipeline).

**TASK**: Please submit your result (i.e., produced text) on Moodle and answer questions about the code.

In [ ]:
# note: if you are running the code on Colab, you may need to install the HuggingFace 'transformers' library
# for that, uncomment and run the following line:

# !pip install transformers

In [ ]:
# import huggingface
from transformers import pipeline

In [ ]:
generator = pipeline(
    'text-generation', 
    model='EleutherAI/gpt-neo-1.3B'
)

### YOUR CODE HERE ###

### Exercise 1.2 (15 points)

Your task is to complete the code below in order to *fine-tune* the model for question answering on the ["Truthful QA" dataset](https://huggingface.co/datasets/truthful_qa).
The goal of this exercise is to understand the basic components that go into fine-tuning an LM from first-hand experience. Therefore, you can run the fine-tuning just for a couple of training steps. 

For convenience, the data loading process is already implemented for you.
You can find relevant information for completing the task [here](https://huggingface.co/transformers/v3.3.1/training.html#fine-tuning-in-native-pytorch).

**TASK**: Please post the code from the cell where you completed something on Moodle. Please answer questions about the other parts of the code on Moodle.

In [ ]:
# first, we import the necessary libraries
# again, use !pip install ... if libraries are missing on Colab
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# load the dataset
dataset = load_dataset("truthful_qa", "generation")

In [ ]:
# inspect a sample from the dataset to get an idea of the formatting
print(dataset['validation'][0])
# the dataset only has a 'validation' split, so we use that. 
# for simplicity, we are not further splitting the data into train/val/test
# but just using everything for training
dataset_val = dataset['validation']

In [ ]:
# load pretrained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")
# add padding token to tokenizer
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# create a pytorch dataset wrapper around the huggingface dataset
# which will allow for easy preprocessing and formatting
class TruthfulQADataset(Dataset):
    """
    Helper class to create a pytorch dataset.
    Each sample if formatted with 'Question: {question} Answer:' prefixes.
    Also pads and truncates the strings to a given maximum length,
    so that they can be batched.
    The implemented methods are required by pytorch.

    Parameters
    ----------
    dataset : huggingface dataset
        The dataset to wrap around.
    tokenizer : huggingface tokenizer
        The tokenizer to use for tokenization.
    max_length : int
        The maximum length of the input and output sequences.
    """
    def __init__(self, dataset, tokenizer, max_length=128):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        """
        Returns a single preprocessed sample from the dataset,
        at given index idx.
        """
         # NOTE: this is an updated version, compared to the initial homework
        # source code. 
        # In particular, it includes correct attention masking and input formatting.
        item = self.dataset[idx]
        question = item['question']
        answer = item['best_answer']
        # format input
        input = f"Question: {question} Answer: {answer}"

        # tokenize input
        # note that the tokenizer automatically adds SOS, EOS and PAD tokens
        inputs = self.tokenizer(
            input, 
            return_tensors='pt', 
            max_length=self.max_length, 
            padding='max_length', 
            truncation=True
        )
        
        return inputs

In [ ]:
# instantiate dataset
train_dataset = TruthfulQADataset(dataset_val, tokenizer)
# create a DataLoader for the dataset
# the data loader will automatically batch the data
# and iteratively return training examples (question answer pairs) in batches
dataloader = DataLoader(
    train_dataset, 
    batch_size=8, 
    shuffle=True
)

In [ ]:
# trianing configutations 
# feel free to play around with these
epochs  = 1
train_steps =  100
# using the GPU if available
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
# put the model in training mode
model.train()
# move the model to the device (e.g. GPU)
model = model.to(device)

# define optimizer and learning rate
optimizer = ### YOUR CODE HERE ###

# define some variables to accumulate the losses
losses = []

# iterate over epochs
for e in range(epochs):
    # iterate over training steps
    for i in range(train_steps):
        # get a batch of data
        x = next(iter(dataloader))
        # move the data to the device (GPU)
        x = x.to(device)

        # forward pass through the model
        ### YOUR CODE HERE ###
        # get the loss
        loss = ### YOUR CODE HERE ###
        # backward pass
        loss.backward()
        losses.append(loss.item())
        # update the parameters of the model
        ### YOUR CODE HERE ###


**NOTE:** The purpose of this exercise is to just get the training running correctly. The quality of the predicted answer after the fine-tuning does not matter for the grading. That is, you don't need to worry in case the predicted answer seems not great to you. 

In [ ]:
# Test the model

# set it to evaluation mode
model.eval()
model = model.to("cpu")
# generate some text for one of the questions from the dataset
question = dataset_val[-1]['question']
print("Question: ", question)
# tokenize the question and generate an answer
input = f"Question: {question} Answer:"
input_ids = tokenizer.encode(input, return_tensors='pt').to('cpu')
prediction = ### YOUR CODE HERE ###
# decode the prediction
answer = tokenizer.decode(prediction[0])
print("Predicted answer after fine-tuning: ", answer)

In [ ]:
# Plot the fine-tuning loss

plt.plot(losses)
plt.xlabel("Training steps")
plt.ylabel("Loss")

## Exercise 2 (10 points)

The goal of this exercise is to apply basic concepts of reinforcement learning to one of the "holy grail" tasks in machine learning and AI -- chess.

Your task is to map concepts like "agent", "action", and "state" that we have discussed in class onto their "real-world" counterparts in the game of chess (e.g., played by a computer program). 

**TASK:** Please fill in your responses on Moodle.

## Exercise 3 (20 points) 

In this exercise, you will train your very first RL agent! 

Imagine that your agent just moved to a new town and is exploring the local restaurants. There are 10 restaurants with the names 0, 1, ..., 9 in this town.
The agent does not know anything about the restaurants in the beginning (and also mysteriously cannot find any reviews to look at). Therefore, she needs to try the restaurants herself and try to figure out which one will make her the happiest during her time in this town (i.e., will give her the highest expected reward). 

This problem of trying to choose which action (i.e., going to which restaurant) is reward-maximizing in one situation, given several action options, is a (simplified) instance of a the so-called *k-armed bandit problem* (where *k* is the number of avialable actions, here: 10). This problem is very well-studied in RL.

For this exercise, we assume a number of simplifications. We assume that the quality of the restaurants is deterministic (i.e., doesn't change over the times the agents goes there), and the agent's preferences don't change over time, either. (Hint: what does this mean for the value of actions and the rewards?)

Based on these assumptions, in this exercise, we apply a simple algorithm estimating the *values of the available actions* $a \in A$ at time $t$ (think: subjective value of going to the restaurant for the agent, e.g., degree of feeling happy upon eating there): 

$$Q_t(a)$$

Specifically, we will apply a simple **sample-average** method for estimating the value of actions at time $t$ wherein the action-values are estimated as averages of the rewards that were received in the past for choosing the respective actions:

$$ Q_t(a) = \frac{\sum_{i=1}^{t-1} R_{i | A_i = a}}{\sum_{i=1}^{t-1} \mathbb{1}_{A_i = a}}$$

Time $t$ here refers to the t-th time the agent is deciding which restaurant to go to in this new town. 
Based on the estimated action values, we will derive two different 'strategies of behavior' (i.e., *policies*): the greedy and the $\epsilon$-greedy policy. 

**TASK:** Your task is to complete the code below to train the agent to explore the restaurants and answer some questions about the results on Moodle.

In [ ]:
# import libraries

import pandas as pd
import numpy as np
np.random.seed(0)

For this $k$-armed restaurant bandit environment, we assume that there is a ground truth value of each of the restaurants for the agent. For instance, we know that the agent's favorite food is Thai curry, and e.g. restaurant 4 has the best Thai curry in town -- therefore, restaurant 4 would have the highest true value. Formally, the true value of an action is:

$$ q_*(a) = \mathbb{E}[R_t \mid A_t = a] $$

On the other hand, the values of the other restaurants might be lower, or even negative (e.g., the agent gets food-poisoning when going there). For tpur simulation, these true values are defined below. These ground truth values are initially unknown to our agent, and her task is to estimate them from experience.

In [ ]:
# define possible actions (10 restaurant in the new town)
actions = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# sample the true values of the actions for the agent
true_restaurant_rewards = np.random.normal(0, 1, 10)
true_restaurant_rewards

In our toy world, the agent tries the different restaurants for multiple days and receives a reward (e.g., writes down her subjective happiness value) every time she went to a restaurant. A single trial is generated by the environment below:

In [ ]:
def town_environment(action, true_restaurant_rewards):
    """
    The town environment returns 'an experience' of our agent,
    i.e., the reward associated with a given action.
    """
    reward = true_restaurant_rewards[action]
    return reward


Your task is to complete the code below so as to implement the estimation of the action values based on past experiences. In particular, your task is to implement an algorith estimating the values of actions based on accumulating experience, and track the expected reward (i.e., mean reward) that the agent would receive if she behaved according to her estimates given the particular amount of experience. 

Specifically, the function below should implement the *sample-average* estimation (defined above) and return an action according to the current estimate. For action selection, please implement two policies:
* a greedy policy (returning the action with the highest value according to the current estimate)
* an $\epsilon$-greedy policy (returning the action with the highest value according to the current estimate in 1-$\epsilon$ proportions of the decisions, and returning a randomly chosen action in $\epsilon$ proportion of cases). You are free to choose your own value of $\epsilon$.

In [ ]:
def sample_average_estimator(old_actions, old_rewards, actions, epsilon=0):
    """
    Implement the sample-average estimator of the action values.

    Parameters
    ----------
    old_actions : numpy array
        The actions taken by the agent before the current step.
    old_rewards : numpy array
        The rewards received by the agent before the current step
        for the taken actions.
    epsilon: float
        The probability of taking a random action.

    Returns
    -------
    values : numpy array
        The estimated values of the actions.
    best_action : int
        The best action according to the estimated values and the 
        current policy.
    """
    # initially, the values of all actions are 0
    values = np.array(###YOUR CODE HERE###)
    # compute averages over previously observed rewards 
    # for each action
    for action in actions:
        old_indices = np.where(old_actions == action)[0]
        if len(old_indices) == 0:
            value = 0
        else:
            ###YOUR CODE HERE###
        values[action] = ###YOUR CODE HERE###
    # return a random action with probability epsilon
    if np.random.uniform(0, 1) < epsilon:
        ###YOUR CODE HERE###
    # return the action with the highest value with random tie-breaking with probability 1 - epsilon
    else:
        if np.sum(values == np.max(values)) > 1:
            best_action = np.random.choice(np.where(values == np.max(values))[0], 1)[0]
        else:
            ###YOUR CODE HERE###
            
    # return the actions' updated values
    # and best action
    return values, best_action


The following cell embeds the function into a loop where the agent gathers experiences over 90 days (i.e., over 90 action-reward pairs) and we can observe how her average reward as well as her action choices develop with accumulated experience.

In [ ]:
# initialize the algorithm
old_actions = np.array([])
old_rewards = np.array([])
# initialize some variables for logging
actions_log = []
rewards_list = []
average_rewards_list = []
# itentify the ground truth optimal action so as to check
# how often the agent would choose it
optimal_action = actions[np.argmax(true_restaurant_rewards)]

# iterate over 90 "experience steps"
for i in range(90):
    # run the algorithm with a GREEDY policy
    
    # return selected action according to current estimates
    values, best_action = ### YOUR CODE HERE ###
    # observe the reward for the currently estimated best action
    reward = town_environment(best_action,true_restaurant_rewards)

    # create experience arrays
    old_actions = np.append(old_actions, best_action)
    old_rewards = np.append(old_rewards, reward)

    # log the results
    # check if the best action is the optimal action
    actions_log.append(best_action == optimal_action)
    rewards_list.append(reward)
    average_rewards_list.append(sum(rewards_list) / len(rewards_list))
    

In [ ]:
# plot results

plt.plot(np.cumsum(actions_log) / np.arange(1, len(actions_log) + 1))
plt.xlabel("Experience steps")
plt.ylabel("Optimal action rate")

In [ ]:
plt.plot(average_rewards_list)
plt.xlabel("Experience steps")
plt.ylabel("Average reward")

In [ ]:
# NOW RUN THE SAME ALGORITHM WITH EPSILON-GREEDY POLICY

### YOUR CODE HERE ###